In [1]:
import os
from importlib import import_module
import copy 
import pandas as pd
import sys
import cobra
import extFunc

In [ ]:
#Load Models if not already in RAM

#nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml', 'iJN1462_GLN_UR5_1.xml', 'iJN1462_GLC_UR7_3.xml' , 'iJN1462_OCT_UR3_4.xml']

nameList = ['iJN1462_GLC_UR10_9.xml' , 'iJN1462_GLC_UR6_3.xml']
if 'modelDict' not in locals():
    modelDict = extFunc.ImportFunction(nameList)   
modelDict.keys()


In [9]:

class ModelComparison():
    '''
    This Class can be used to compare multiple modelversions 
    and to get an overview of the C-Flux routes and 
    the corresponding fva
    input  = list of models
    '''
    def __init__( self, modelDict: "modelDict", **kwargs ):
        self.modelDict     = modelDict
        self.solutionDict = {}
        self.summaryDict   = {}
        self.dataFrameDict = {}
        self.generate()
            
    #collect the objects of interest
    def generate(self):
        for model in self.modelDict.keys():
            self.solutionDict[model]    = self.modelDict[model].optimize()
            self.summaryDict[model]     = self.modelDict[model].summary(solution=self.solutionDict[model], fva=.99)
            self.dataFrameDict[model]   = {
                                            "uptake"     :   pd.read_html(self.summaryDict[model].to_html())[0].sort_values('C-Flux',ascending=False),
                                            "seecretion" :   pd.read_html(self.summaryDict[model].to_html())[1].sort_values('C-Flux',ascending=False)
                                           }
    
    def mySummary(self):
        df2 = pd.DataFrame(
            {
    "A": self.summaryDict[model].fluxes.getby,
    "B": pd.Timestamp("20130102"),
    "C": pd.Series(1, index=list(range(4)), dtype="float32"),
    "E": pd.Categorical(["test", "train", "test", "train"]),
    "F": "foo",
            }
        )
    return df2
          

if 'comp' not in locals():
    comp = ModelComparison(modelDict=modelDict)


In [64]:
comp
'''
def createOutputWrapper(modelList):

    pass
    for model in modelList:
        tableForDisppay.join(modelList)      

    return tableForDisppay

def createOutput(model):
    sol = model.optimize()
    sum = model.summary(solution = sol,fva=.99)
    
    #biomass = solution.reac ('BIOMASS_KT2440_WT3') 
    print(biomass)
'''
    

#createOutput(modelList)       
#das gesamte Paket als 
#df = iJN1462_OCT_UR3_4.summary().to_frame()

#iJN1462_OCT_UR3_4.tolerance = 1e-1
#iJN1462_OCT_UR3_4.summary()
#dhtml =de.to_html()
#df = pd.read_html(dhtml)
#pd.DataFrame(df[1]).sort_values('C-Flux',ascending=False)
#df[1]
#
#de
#iJN1462_OCT_UR3_4.summary()
#iJN1462_OCT_UR3_4._tolerance = 2
#solutionSummaryObject = iJN1462_OCT_UR3_4.summary(fva=.99)


#pd.DataFrame(pd.read_html(dhtml)[1]).sort_values('C-Flux',ascending=False)


#iJN1462_OCT_UR3_4.reactions.get_by_id('biomass')
#createOutput(iJN1462_GLC_UR10_9)
comp.modelDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_glc__D_e')
comp.modelDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_octa_e')
comp.modelDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_glc__D_e')
comp.modelDict['iJN1462_GLC_UR10_9'].fluxes.get_by_id('EX_octa_e')

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2[e],EX_ca2_e,0.002422,0,0.00%
cl[e],EX_cl_e,0.002422,0,0.00%
cobalt2[e],EX_cobalt2_e,0.001742,0,0.00%
cu2[e],EX_cu2_e,0.001614,0,0.00%
fe2[e],EX_fe2_e,0.008417,0,0.00%
glc__D[e],EX_glc__D_e,10.9,6,100.00%
k[e],EX_k_e,0.09084,0,0.00%
mg2[e],EX_mg2_e,0.004037,0,0.00%
mn2[e],EX_mn2_e,0.001614,0,0.00%
mobd[e],EX_mobd_e,0.00187,0,0.00%
